<!-- Import packages -->

## Install and import

Install packages

In [246]:
pip install -r Requirements.txt

ERROR: Could not find a version that satisfies the requirement python==3.11 (from versions: none)
ERROR: No matching distribution found for python==3.11
Note: you may need to restart the kernel to use updated packages.


Import libraries


In [247]:
import pandas as pd
import numpy as np
import matplotlib as plt
import json
import datetime as dt
import requests
import os
from fredapi import Fred

## S&P500 data extract and clean

Create a date range. This date range has been set to 1950-present because volume data is missing before 1950.

In [248]:
end_date = dt.datetime.now()
start_date = dt.datetime(1950, 1, 1)

Pull S&P500 data using API from rapidapi.com

In [249]:
url = "https://s-p-500-history-api.p.rapidapi.com/historical-stock-data.php"

querystring = {"interval":"day","symbol":"^GSPC","start":start_date,"end":end_date}

headers = {
	"x-rapidapi-key": "1818711435msh7ff1e71cfe61ec4p1b64e1jsn32c289b7a487",
	"x-rapidapi-host": "s-p-500-history-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


Extract data from json file. Daily and weekly timeframes. My API monthly data only went back to 1985.

In [250]:
response_json = json.loads(response.text)
sp = pd.DataFrame(response_json['data'])
sp.head()

querystring = {"interval":"week","symbol":"^GSPC","start":start_date,"end":end_date}
response = requests.get(url, headers=headers, params=querystring)
response_json = json.loads(response.text)
sp_weekly = pd.DataFrame(response_json['data'])
sp_weekly.head()

,date_time,timestamp,high,low,close,open,volume,adjusted_close
0,"January 1, 1950, 5:00 AM UTC",-631134000,17.090000,16.660000,17.090000,16.660000,9040000,17.090000
1,"January 8, 1950, 5:00 AM UTC",-630529200,17.090000,16.650000,16.650000,17.080000,14790000,16.650000
2,"January 15, 1950, 5:00 AM UTC",-629924400,16.940001,16.719999,16.940001,16.719999,7980000,16.940001
3,"January 22, 1950, 5:00 AM UTC",-629319600,16.920000,16.730000,16.900000,16.920000,7430000,16.900000
4,"January 29, 1950, 5:00 AM UTC",-628714800,17.350000,17.020000,17.350000,17.020000,10120000,17.350000


Edit 'date_time' field in sp and sp_weekly to datetime dtype

In [251]:
sp['date_time'].info

<bound method Series.info of 0        January 3, 1950, 2:30 PM UTC
1        January 4, 1950, 2:30 PM UTC
2        January 5, 1950, 2:30 PM UTC
3        January 6, 1950, 2:30 PM UTC
4        January 9, 1950, 2:30 PM UTC
                     ...             
18938     April 10, 2025, 1:30 PM UTC
18939     April 11, 2025, 1:30 PM UTC
18940     April 14, 2025, 1:30 PM UTC
18941     April 15, 2025, 1:30 PM UTC
18942     April 16, 2025, 1:30 PM UTC
Name: date_time, Length: 18943, dtype: object>

In [252]:
sp_weekly['date_time'] = pd.to_datetime(sp_weekly['date_time'])
sp['date_time'] = pd.to_datetime(sp['date_time'])

/tmp/ipykernel_45126/4117627709.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sp['date_time'] = pd.to_datetime(sp['date_time'])


In [253]:
sp['date_time'].info

<bound method Series.info of 0       1950-01-03 14:30:00+00:00
1       1950-01-04 14:30:00+00:00
2       1950-01-05 14:30:00+00:00
3       1950-01-06 14:30:00+00:00
4       1950-01-09 14:30:00+00:00
                   ...           
18938   2025-04-10 13:30:00+00:00
18939   2025-04-11 13:30:00+00:00
18940   2025-04-14 13:30:00+00:00
18941   2025-04-15 13:30:00+00:00
18942   2025-04-16 13:30:00+00:00
Name: date_time, Length: 18943, dtype: datetime64[ns, UTC]>

Drop 'timestamp' and 'adjusted close' columns

In [254]:
sp.drop(columns=['timestamp','adjusted_close'], inplace=True, axis=1)

sp_weekly.drop(columns=['timestamp','adjusted_close'], inplace=True, axis=1)

## Collect macroeconomic FRED data

Extract urls from 'FRED Data URLs.json' (exported file of FRED webpages I am going to pull from) using 'cntrl+shift+L' in VSCode

In [255]:

url_list = [
"https://fred.stlouisfed.org/series/PAYEMS",
"https://fred.stlouisfed.org/series/INDPRO",
"https://fred.stlouisfed.org/series/CE16OV",
"https://fred.stlouisfed.org/series/UNRATE",
"https://fred.stlouisfed.org/series/GDP",
"https://fred.stlouisfed.org/series/A191RP1Q027SBEA",
"https://fred.stlouisfed.org/series/CPIAUCSL",
"https://fred.stlouisfed.org/series/SAHMCURRENT",
"https://fred.stlouisfed.org/series/LORSGPORUSQ659S",
"https://fred.stlouisfed.org/series/BOGZ1FL135010005Q",
"https://fred.stlouisfed.org/series/UMCSENT",
"https://fred.stlouisfed.org/series/HOUST",
"https://fred.stlouisfed.org/series/W875RX1#0##0",
"https://fred.stlouisfed.org/series/M2SL",
"https://fred.stlouisfed.org/series/PRS85006091",
"https://fred.stlouisfed.org/series/DFF",
"https://fred.stlouisfed.org/series/SHTSAUS",
"https://fred.stlouisfed.org/series/M1V",
"https://fred.stlouisfed.org/series/M2V",
"https://fred.stlouisfed.org/series/INTDSRUSM193N"
]


Use BeautifulSoup to extract metadata from above urls. Create a dictionary and lists of those values.

In [256]:
from bs4 import BeautifulSoup

fred_dictionary = {}
title_list = []
notes_list = []
id_list = []
frequency_list = []

# Loop through the URLs and extract the title and paragraph under "Notes"
for url in url_list:
# Extract the title 
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    title = soup.title.string.strip()
    title_list.append(title)
# Extract the paragraph under "Notes"
    notes = soup.find("div", id="notes") 
    notes_text = notes.get_text(strip=True)
    notes_list.append(notes_text) 
# Extract the "Frequency"
    frequency = soup.find("span", class_="series-meta-value-frequency")
    frequency_text = frequency.get_text(strip=True)
    frequency_list.append(frequency_text) 
# Extract the ID values from () in the title
    pipe = title.find("|")
    left_of_pipe = title[:pipe]
    id = left_of_pipe[left_of_pipe.rfind("("):]
    id = id.replace("(", "")
    id = id.replace(")", "")
    id_text = id.strip() 
    id_list.append(id_text)

# Create a dictionary with the extracted data
    fred_dictionary[id_text] = {
        "url": url,
        "id": id_text,
        "title": title,
        "frequency": frequency_text,
        "notes": notes_text
    }

# Print the extracted data
    print(f"URL: {url}")
    print(f"ID: {id_text}")
    print(f"Title: {title}") 
    print(f"Frequency: {frequency_text}") 
    print(f"Notes: {notes_text}\n")


URL: https://fred.stlouisfed.org/series/PAYEMS
ID: PAYEMS
Title: All Employees, Total Nonfarm (PAYEMS) | FRED | St. Louis Fed
Frequency: Monthly
Notes: NotesSource:U.S. Bureau of Labor StatisticsRelease:Employment SituationUnits:Thousands of Persons, Seasonally AdjustedFrequency:MonthlyNotes:All Employees: Total Nonfarm, commonly known as Total Nonfarm Payroll, is a measure of the number of U.S. workers in the economy that excludes proprietors, private household employees, unpaid volunteers, farm employees, and the unincorporated self-employed. This measure accounts for approximately 80 percent of the workers who contribute to Gross Domestic Product (GDP).This measure provides useful insights into the current economic situation because it can represent the number of jobs added or lost in an economy. Increases in employment might indicate that businesses are hiring which might also suggest that businesses are growing. Additionally, those who are newly employed have increased their perso

Use the FRED API library and use a key to access series. Make a dictionary out of those series.

In [257]:
# FRED API
fred_api_key = Fred(api_key='9de274aca217f6c8787ddb88fbcf323c')

df_dict = {}

for df_name in id_list:
    fred_series = fred_api_key.get_series(df_name, start_date, end_date)
    df_dict[df_name] = pd.DataFrame(fred_series, columns=[df_name])
df_dict

{'PAYEMS':               PAYEMS
 1950-01-01   43526.0
 1950-02-01   43297.0
 1950-03-01   43954.0
 1950-04-01   44382.0
 1950-05-01   44718.0
 ...              ...
 2024-11-01  158619.0
 2024-12-01  158942.0
 2025-01-01  159053.0
 2025-02-01  159170.0
 2025-03-01  159398.0
 
 [903 rows x 1 columns],
 'INDPRO':               INDPRO
 1950-01-01   14.0049
 1950-02-01   14.0586
 1950-03-01   14.5156
 1950-04-01   14.9995
 1950-05-01   15.3489
 ...              ...
 2024-11-01  101.9619
 2024-12-01  103.1177
 2025-01-01  103.3418
 2025-02-01  104.2202
 2025-03-01  103.8892
 
 [903 rows x 1 columns],
 'CE16OV':               CE16OV
 1950-01-01   57635.0
 1950-02-01   57751.0
 1950-03-01   57728.0
 1950-04-01   58583.0
 1950-05-01   58649.0
 ...              ...
 2024-11-01  161183.0
 2024-12-01  161661.0
 2025-01-01  163895.0
 2025-02-01  163307.0
 2025-03-01  163508.0
 
 [903 rows x 1 columns],
 'UNRATE':             UNRATE
 1950-01-01     6.5
 1950-02-01     6.4
 1950-03-01     6.3
 1950-0

## Clean FRED DataFrame Dictionary

Make a copy of the df_dict

In [258]:
df_dict_copy = df_dict.copy()

Check the columns of dataframes in dictionary  

In [259]:
for key, value in df_dict_copy.items():
    print(f"{key}: {value.columns}")

PAYEMS: Index(['PAYEMS'], dtype='object')
INDPRO: Index(['INDPRO'], dtype='object')
CE16OV: Index(['CE16OV'], dtype='object')
UNRATE: Index(['UNRATE'], dtype='object')
GDP: Index(['GDP'], dtype='object')
A191RP1Q027SBEA: Index(['A191RP1Q027SBEA'], dtype='object')
CPIAUCSL: Index(['CPIAUCSL'], dtype='object')
SAHMCURRENT: Index(['SAHMCURRENT'], dtype='object')
LORSGPORUSQ659S: Index(['LORSGPORUSQ659S'], dtype='object')
BOGZ1FL135010005Q: Index(['BOGZ1FL135010005Q'], dtype='object')
UMCSENT: Index(['UMCSENT'], dtype='object')
HOUST: Index(['HOUST'], dtype='object')
W875RX1: Index(['W875RX1'], dtype='object')
M2SL: Index(['M2SL'], dtype='object')
PRS85006091: Index(['PRS85006091'], dtype='object')
DFF: Index(['DFF'], dtype='object')
SHTSAUS: Index(['SHTSAUS'], dtype='object')
M1V: Index(['M1V'], dtype='object')
M2V: Index(['M2V'], dtype='object')
INTDSRUSM193N: Index(['INTDSRUSM193N'], dtype='object')


Loop through to rename the first column to 'date' and set is as datetime

In [260]:
for key, value in df_dict_copy.items():
    df_dict_copy[key] = df_dict_copy[key].reset_index()
    df_dict_copy[key].rename(columns={"index": "date"}, inplace=True) 
    df_dict_copy[key]['date'] = pd.to_datetime(df_dict_copy[key]['date']) 
    df_dict_copy[key].set_index('date', inplace=True)



Look for NAs

In [261]:
for key, value in df_dict_copy.items():
    if value.isna().sum().sum() > 0:
        print(f"{key}: {value.isna().sum()}")


BOGZ1FL135010005Q: BOGZ1FL135010005Q    6
dtype: int64
UMCSENT: UMCSENT    210
dtype: int64


In [262]:
print(df_dict_copy['BOGZ1FL135010005Q'][df_dict_copy['BOGZ1FL135010005Q'].isna().any(axis=1)])

print(df_dict_copy['UMCSENT'][df_dict_copy['UMCSENT'].isna().any(axis=1)])

            BOGZ1FL135010005Q
date                         
1950-01-01                NaN
1950-04-01                NaN
1950-07-01                NaN
1951-01-01                NaN
1951-04-01                NaN
1951-07-01                NaN
            UMCSENT
date               
1952-12-01      NaN
1953-01-01      NaN
1953-03-01      NaN
1953-04-01      NaN
1953-05-01      NaN
...             ...
1977-06-01      NaN
1977-07-01      NaN
1977-09-01      NaN
1977-10-01      NaN
1977-12-01      NaN

[210 rows x 1 columns]


Drop UMCSENT from dictionary and clean the BOGZ1FL135010005Q dataframe

In [263]:
df_dict_copy['BOGZ1FL135010005Q'].dropna(inplace=True)
df_dict_copy.pop('UMCSENT')

# Drop the first row of the DataFrame
df_dict_copy['BOGZ1FL135010005Q'] = df_dict_copy['BOGZ1FL135010005Q'].iloc[1:]

df_dict_copy['BOGZ1FL135010005Q'].head()

,BOGZ1FL135010005Q
date,
1951-10-01,66460.0
1952-01-01,66604.0
1952-04-01,66759.0
1952-07-01,66934.0
1952-10-01,67027.0


Identify all monthly datasets and quarterly datasets

In [264]:
for key, value in fred_dictionary.items():
    print(f"{key}: {value['frequency']}")




PAYEMS: Monthly
INDPRO: Monthly
CE16OV: Monthly
UNRATE: Monthly
GDP: Quarterly
A191RP1Q027SBEA: Quarterly
CPIAUCSL: Monthly
SAHMCURRENT: Monthly
LORSGPORUSQ659S: Quarterly
BOGZ1FL135010005Q: Quarterly
UMCSENT: Monthly
HOUST: Monthly
W875RX1: Monthly
M2SL: Monthly
PRS85006091: Quarterly
DFF: Daily,7-Day
SHTSAUS: Monthly
M1V: Quarterly
M2V: Quarterly
INTDSRUSM193N: Monthly


Review DFF

In [265]:
df_dict_copy['DFF'].head()
#This is all good and will be added as daily

,DFF
date,
1954-07-01,1.13
1954-07-02,1.25
1954-07-03,1.25
1954-07-04,1.25
1954-07-05,0.88


Separate them into new monthly and quarterly dictionaries

In [266]:
monthly_dfs = {}
quarterly_dfs = {}

# Use the key to look up the frequency in fred_dictionary
for key, value in df_dict_copy.items():
    if fred_dictionary[key]['frequency'] == "Monthly":
        monthly_dfs[key] = value
    elif fred_dictionary[key]['frequency'] == "Quarterly":
        quarterly_dfs[key] = value

## Convert monthly and quarterly dictionaries into two DataFrames, then combine into single daily DataFrame

Create dataframes for monthly and quarterly, then combine dataframes in dictionaries

In [267]:
combined_monthly_dfs = pd.concat(monthly_dfs.values(), axis=1)

combined_quarterly_dfs = pd.concat(quarterly_dfs.values(), axis=1)

Review NAs to see which dates are missing from which feature. I've commented out the max_rows option because it was slowing down my system and I only needed to look at all values once.

In [268]:
# pd.set_option('display.max_rows', None)

combined_quarterly_dfs[combined_quarterly_dfs.isna().any(axis=1)]
combined_monthly_dfs[combined_monthly_dfs.isna().any(axis=1)]

,PAYEMS,INDPRO,CE16OV,UNRATE,CPIAUCSL,SAHMCURRENT,HOUST,W875RX1,M2SL,SHTSAUS,INTDSRUSM193N
date,,,,,,,,,,,
1950-01-01,43526.0,14.0049,57635.0,6.5,23.510,2.47,NaN,NaN,NaN,NaN,1.5
1950-02-01,43297.0,14.0586,57751.0,6.4,23.610,2.17,NaN,NaN,NaN,NaN,1.5
1950-03-01,43954.0,14.5156,57728.0,6.3,23.640,1.73,NaN,NaN,NaN,NaN,1.5
1950-04-01,44382.0,14.9995,58583.0,5.8,23.650,1.17,NaN,NaN,NaN,NaN,1.5
1950-05-01,44718.0,15.3489,58649.0,5.5,23.770,0.40,NaN,NaN,NaN,NaN,1.5
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-01,158619.0,101.9619,161183.0,4.2,316.449,0.40,1305.0,16376.8,21454.5,109.0,NaN
2024-12-01,158942.0,103.1177,161661.0,4.1,317.603,0.40,1526.0,16387.7,21490.1,104.0,NaN
2025-01-01,159053.0,103.3418,163895.0,4.0,319.086,0.37,1350.0,16398.1,21577.1,105.0,NaN


#### Quarterly: 
M1V, M2V start in 1959. I'm just going to keep those attributes and remove those data entries. 

LORSGPORUSQ659S ends in 2024. I'm going to remove this feature as this is just a composite of leading indicators assembled by the FED, so all info should be in the other features.

#### Monthly: 
HOUST, W875RX1, M2SL, SHTSAUS all start in 1958. I'm just going to keep those attributes and remove those data entries. 

INTDSRUSM193N ends in 2022. I'm going to remove this feature and just use DFF for interest rates.

In [269]:
combined_quarterly_dfs.drop(columns=['LORSGPORUSQ659S'], inplace=True)
combined_monthly_dfs.drop(columns=['INTDSRUSM193N'], inplace=True)

combined_quarterly_dfs.dropna(inplace=True)
combined_monthly_dfs.dropna(inplace=True)

Data looks good now! 1959-present now

In [270]:
combined_quarterly_dfs
combined_monthly_dfs

,PAYEMS,INDPRO,CE16OV,UNRATE,CPIAUCSL,SAHMCURRENT,HOUST,W875RX1,M2SL,SHTSAUS
date,,,,,,,,,,
1959-01-01,52478.0,21.9616,63868.0,6.0,29.010,0.77,1657.0,2426.0,286.6,113.0
1959-02-01,52688.0,22.3917,63684.0,5.9,29.000,0.23,1667.0,2434.8,287.7,115.0
1959-03-01,53014.0,22.7142,64267.0,5.6,28.970,-0.20,1620.0,2452.7,289.2,126.0
1959-04-01,53321.0,23.1981,64768.0,5.2,28.980,-0.27,1590.0,2470.0,290.1,113.0
1959-05-01,53550.0,23.5476,64699.0,5.1,29.040,-0.27,1498.0,2486.4,292.2,120.0
...,...,...,...,...,...,...,...,...,...,...
2024-10-01,158358.0,102.2138,161456.0,4.1,315.564,0.40,1344.0,16335.8,21335.5,106.0
2024-11-01,158619.0,101.9619,161183.0,4.2,316.449,0.40,1305.0,16376.8,21454.5,109.0
2024-12-01,158942.0,103.1177,161661.0,4.1,317.603,0.40,1526.0,16387.7,21490.1,104.0


Adjust combined_quarterly_dfs and combined_monthly_dfs to daily, then combine them. I've decided to just forward fill, so I'll see if that has any negative impact. Based on research, adding binary flag columns could be useful.

In [271]:
combined_quarterly_dfs_daily = combined_quarterly_dfs.resample('D').ffill()
combined_monthly_dfs_daily = combined_monthly_dfs.resample('D').ffill()

daily = pd.concat([combined_quarterly_dfs_daily, combined_monthly_dfs_daily], axis=1)
daily

,GDP,A191RP1Q027SBEA,BOGZ1FL135010005Q,PRS85006091,M1V,M2V,PAYEMS,INDPRO,CE16OV,UNRATE,CPIAUCSL,SAHMCURRENT,HOUST,W875RX1,M2SL,SHTSAUS
date,,,,,,,,,,,,,,,,
1959-01-01,510.33,8.9,96246.0,5.7,3.663,1.773,52478.0,21.9616,63868.0,6.0,29.010,0.77,1657.0,2426.0,286.6,113.0
1959-01-02,510.33,8.9,96246.0,5.7,3.663,1.773,52478.0,21.9616,63868.0,6.0,29.010,0.77,1657.0,2426.0,286.6,113.0
1959-01-03,510.33,8.9,96246.0,5.7,3.663,1.773,52478.0,21.9616,63868.0,6.0,29.010,0.77,1657.0,2426.0,286.6,113.0
1959-01-04,510.33,8.9,96246.0,5.7,3.663,1.773,52478.0,21.9616,63868.0,6.0,29.010,0.77,1657.0,2426.0,286.6,113.0
1959-01-05,510.33,8.9,96246.0,5.7,3.663,1.773,52478.0,21.9616,63868.0,6.0,29.010,0.77,1657.0,2426.0,286.6,113.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-28,NaN,NaN,NaN,NaN,NaN,NaN,159053.0,103.3418,163895.0,4.0,319.086,0.37,1350.0,16398.1,21577.1,105.0
2025-01-29,NaN,NaN,NaN,NaN,NaN,NaN,159053.0,103.3418,163895.0,4.0,319.086,0.37,1350.0,16398.1,21577.1,105.0
2025-01-30,NaN,NaN,NaN,NaN,NaN,NaN,159053.0,103.3418,163895.0,4.0,319.086,0.37,1350.0,16398.1,21577.1,105.0


Review NAs in daily

In [272]:
daily[daily.isna().any(axis=1)]

,GDP,A191RP1Q027SBEA,BOGZ1FL135010005Q,PRS85006091,M1V,M2V,PAYEMS,INDPRO,CE16OV,UNRATE,CPIAUCSL,SAHMCURRENT,HOUST,W875RX1,M2SL,SHTSAUS
date,,,,,,,,,,,,,,,,
2024-10-02,NaN,NaN,NaN,NaN,NaN,NaN,158358.0,102.2138,161456.0,4.1,315.564,0.40,1344.0,16335.8,21335.5,106.0
2024-10-03,NaN,NaN,NaN,NaN,NaN,NaN,158358.0,102.2138,161456.0,4.1,315.564,0.40,1344.0,16335.8,21335.5,106.0
2024-10-04,NaN,NaN,NaN,NaN,NaN,NaN,158358.0,102.2138,161456.0,4.1,315.564,0.40,1344.0,16335.8,21335.5,106.0
2024-10-05,NaN,NaN,NaN,NaN,NaN,NaN,158358.0,102.2138,161456.0,4.1,315.564,0.40,1344.0,16335.8,21335.5,106.0
2024-10-06,NaN,NaN,NaN,NaN,NaN,NaN,158358.0,102.2138,161456.0,4.1,315.564,0.40,1344.0,16335.8,21335.5,106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-28,NaN,NaN,NaN,NaN,NaN,NaN,159053.0,103.3418,163895.0,4.0,319.086,0.37,1350.0,16398.1,21577.1,105.0
2025-01-29,NaN,NaN,NaN,NaN,NaN,NaN,159053.0,103.3418,163895.0,4.0,319.086,0.37,1350.0,16398.1,21577.1,105.0
2025-01-30,NaN,NaN,NaN,NaN,NaN,NaN,159053.0,103.3418,163895.0,4.0,319.086,0.37,1350.0,16398.1,21577.1,105.0


All quarterly data ends October of 2024. I will just drop these NAs.

In [273]:
daily.dropna(inplace=True)

## Format and combine sp with sp_weekly

Review S&P dataframes

In [274]:
sp
sp_weekly

,date_time,high,low,close,open,volume
0,1950-01-01 05:00:00+00:00,17.090000,16.660000,17.090000,16.660000,9040000
1,1950-01-08 05:00:00+00:00,17.090000,16.650000,16.650000,17.080000,14790000
2,1950-01-15 05:00:00+00:00,16.940001,16.719999,16.940001,16.719999,7980000
3,1950-01-22 05:00:00+00:00,16.920000,16.730000,16.900000,16.920000,7430000
4,1950-01-29 05:00:00+00:00,17.350000,17.020000,17.350000,17.020000,10120000
...,...,...,...,...,...,...
3925,2025-03-23 04:00:00+00:00,5786.950195,5572.419922,5580.939941,5718.080078,22085260000
3926,2025-03-30 04:00:00+00:00,5695.310059,5069.899902,5074.080078,5527.910156,29999550000
3927,2025-04-06 04:00:00+00:00,5481.339844,4835.040039,5363.359863,4953.790039,37869410000
3928,2025-04-13 04:00:00+00:00,5459.459961,5220.790039,5275.700195,5441.959961,13956300000


Change date_time to be dates and index for sp and sp_weekly DataFrames

In [275]:
# Create a new 'date' column to replace 'date_time' 
sp_copy = sp.copy()
sp_copy['date'] = pd.to_datetime(sp_copy['date_time'])
sp_copy['date'] = sp_copy['date'].tz_localize(None)
sp_copy.drop(columns=['date_time'], inplace=True)
sp_copy.set_index('date', inplace=True)

sp_weekly_copy = sp_weekly.copy()
sp_weekly_copy['date'] = pd.to_datetime(sp_weekly_copy['date_time']).dt.normalize()
sp_weekly_copy['date'] = sp_weekly_copy['date'].tz_localize(None)
sp_weekly_copy.drop(columns=['date_time'], inplace=True)
sp_weekly_copy.set_index('date', inplace=True)

TypeError: index is not a valid DatetimeIndex or PeriodIndex

Change sp_weekly to be daily and forward fill

In [ ]:
sp_weekly_copy = sp_weekly_copy.resample('D').ffill()

Remove close and open columns from weekly. And rename high, open, volume to _weekly

In [ ]:
sp_weekly_copy.drop(columns=['low','close'], inplace=True)
sp_weekly_copy

sp_weekly_copy.rename(columns={'high': 'high_weekly'}, inplace=True)
sp_weekly_copy.rename(columns={'open': 'open_weekly'}, inplace=True)
sp_weekly_copy.rename(columns={'volume': 'volume_weekly'}, inplace=True)

In [ ]:
sp_weekly_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 27500 entries, 1950-01-01 00:00:00+00:00 to 2025-04-16 00:00:00+00:00
Freq: D
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   high_weekly    27500 non-null  float64
 1   open_weekly    27500 non-null  float64
 2   volume_weekly  27500 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 859.4 KB


Reindex sp to be business weekdays to all calendar days like the other DataFrames

In [ ]:
sp_copy = sp_copy.reindex(sp_weekly_copy.index, method='ffill')


In [ ]:
sp_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 27500 entries, 1950-01-01 00:00:00+00:00 to 2025-04-16 00:00:00+00:00
Freq: D
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   high    27498 non-null  float64
 1   low     27498 non-null  float64
 2   close   27498 non-null  float64
 3   open    27498 non-null  float64
 4   volume  27498 non-null  float64
dtypes: float64(5)
memory usage: 1.3 MB


Merge the sp DataFrames

In [ ]:
sp_final = pd.concat([sp_copy, sp_weekly_copy], axis=1)
sp_final.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 27500 entries, 1950-01-01 00:00:00+00:00 to 2025-04-16 00:00:00+00:00
Freq: D
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   high           27498 non-null  float64
 1   low            27498 non-null  float64
 2   close          27498 non-null  float64
 3   open           27498 non-null  float64
 4   volume         27498 non-null  float64
 5   high_weekly    27500 non-null  float64
 6   open_weekly    27500 non-null  float64
 7   volume_weekly  27500 non-null  int64  
dtypes: float64(7), int64(1)
memory usage: 1.9 MB


Check the NA values

In [ ]:
sp_final[sp_final.isna().any(axis=1)]


,high,low,close,open,volume,high_weekly,open_weekly,volume_weekly
date,,,,,,,,
1950-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,17.09,16.66,9040000
1950-01-02 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,17.09,16.66,9040000


Backfill those two NAs

In [ ]:
sp_final.fillna(method='bfill', inplace=True)
sp_final

/tmp/ipykernel_45126/1730702201.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sp_final.fillna(method='bfill', inplace=True)


,high,low,close,open,volume,high_weekly,open_weekly,volume_weekly
date,,,,,,,,
1950-01-01 00:00:00+00:00,16.660000,16.660000,16.660000,16.660000,1.260000e+06,17.090000,16.660000,9040000
1950-01-02 00:00:00+00:00,16.660000,16.660000,16.660000,16.660000,1.260000e+06,17.090000,16.660000,9040000
1950-01-03 00:00:00+00:00,16.660000,16.660000,16.660000,16.660000,1.260000e+06,17.090000,16.660000,9040000
1950-01-04 00:00:00+00:00,16.850000,16.850000,16.850000,16.850000,1.890000e+06,17.090000,16.660000,9040000
1950-01-05 00:00:00+00:00,16.930000,16.930000,16.930000,16.930000,2.550000e+06,17.090000,16.660000,9040000
...,...,...,...,...,...,...,...,...
2025-04-12 00:00:00+00:00,5381.459961,5220.770020,5363.359863,5255.560059,5.602550e+09,5481.339844,4953.790039,37869410000
2025-04-13 00:00:00+00:00,5381.459961,5220.770020,5363.359863,5255.560059,5.602550e+09,5459.459961,5441.959961,13956300000
2025-04-14 00:00:00+00:00,5459.459961,5358.020020,5405.970215,5441.959961,5.031440e+09,5459.459961,5441.959961,13956300000


## Combine sp_final DataFrame with daily DataFrame and DFF

In [279]:
sp_final = pd.concat([sp_final, daily], axis=1)
sp_final

,high,low,close,open,volume,high_weekly,open_weekly,volume_weekly,GDP,A191RP1Q027SBEA,...,PAYEMS,INDPRO,CE16OV,UNRATE,CPIAUCSL,SAHMCURRENT,HOUST,W875RX1,M2SL,SHTSAUS
date,,,,,,,,,,,,,,,,,,,,,
1950-01-01,16.660000,16.660000,16.660000,16.660000,1.260000e+06,17.090000,16.660000,9040000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-02,16.660000,16.660000,16.660000,16.660000,1.260000e+06,17.090000,16.660000,9040000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-03,16.660000,16.660000,16.660000,16.660000,1.260000e+06,17.090000,16.660000,9040000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-04,16.850000,16.850000,16.850000,16.850000,1.890000e+06,17.090000,16.660000,9040000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-01-05,16.930000,16.930000,16.930000,16.930000,2.550000e+06,17.090000,16.660000,9040000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-12,5381.459961,5220.770020,5363.359863,5255.560059,5.602550e+09,5481.339844,4953.790039,37869410000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-13,5381.459961,5220.770020,5363.359863,5255.560059,5.602550e+09,5459.459961,5441.959961,13956300000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-14,5459.459961,5358.020020,5405.970215,5441.959961,5.031440e+09,5459.459961,5441.959961,13956300000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


sp_final drop all NAs

In [280]:
sp_final.dropna(inplace=True)
sp_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24016 entries, 1959-01-01 to 2024-10-01
Freq: D
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   high               24016 non-null  float64
 1   low                24016 non-null  float64
 2   close              24016 non-null  float64
 3   open               24016 non-null  float64
 4   volume             24016 non-null  float64
 5   high_weekly        24016 non-null  float64
 6   open_weekly        24016 non-null  float64
 7   volume_weekly      24016 non-null  int64  
 8   GDP                24016 non-null  float64
 9   A191RP1Q027SBEA    24016 non-null  float64
 10  BOGZ1FL135010005Q  24016 non-null  float64
 11  PRS85006091        24016 non-null  float64
 12  M1V                24016 non-null  float64
 13  M2V                24016 non-null  float64
 14  PAYEMS             24016 non-null  float64
 15  INDPRO             24016 non-null  float64
 1

Add DFF data to sp_final DataFrame

In [283]:
sp_final = pd.concat([sp_final,df_dict_copy['DFF']], axis=1)
sp_final.dropna(inplace=True)
sp_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24016 entries, 1959-01-01 to 2024-10-01
Freq: D
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   high               24016 non-null  float64
 1   low                24016 non-null  float64
 2   close              24016 non-null  float64
 3   open               24016 non-null  float64
 4   volume             24016 non-null  float64
 5   high_weekly        24016 non-null  float64
 6   open_weekly        24016 non-null  float64
 7   volume_weekly      24016 non-null  float64
 8   GDP                24016 non-null  float64
 9   A191RP1Q027SBEA    24016 non-null  float64
 10  BOGZ1FL135010005Q  24016 non-null  float64
 11  PRS85006091        24016 non-null  float64
 12  M1V                24016 non-null  float64
 13  M2V                24016 non-null  float64
 14  PAYEMS             24016 non-null  float64
 15  INDPRO             24016 non-null  float64
 1